<a href="https://colab.research.google.com/github/dhia-eddine-jedidi/Deep-Learning-Project-using-Autoencoder-CNN-LSTM/blob/main/Deep_Learning_Project_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("stefanogiannini/crema-d-video")

print("Path to dataset files:", path)

100%|██████████| 1.97G/1.97G [01:35<00:00, 22.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1


In [2]:
import os
import subprocess
from concurrent.futures import ThreadPoolExecutor

input_dir = "/root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1"
output_dir = "outputMP4"
os.makedirs(output_dir, exist_ok=True)
MAX_WORKERS = 16  # parallel conversions
def convert_to_mp4(flv_path):
    base = os.path.splitext(os.path.basename(flv_path))[0]
    mp4_path = os.path.join(output_dir, f"{base}.mp4")

    # Re-encode video (H.264) + audio (AAC) → MP4
    result = subprocess.run([
        "ffmpeg", "-y", "-i", flv_path,
        "-c:v", "libx264", "-preset", "fast", "-crf", "23",
        "-c:a", "aac", "-b:a", "128k",
        mp4_path
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if result.returncode == 0:
        print(f"[✅] Converted: {flv_path} → {mp4_path}")
        return True
    else:
        print(f"[❌] Failed: {flv_path}")
        print(result.stderr.decode('utf-8'))
        return False

def convert_all():
    flv_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".flv")]
    if not flv_files:
        print("⚠️ No .flv files found.")
        return

    print(f"🔄 Found {len(flv_files)} .flv files — starting conversion with re-encoding...")
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        list(executor.map(convert_to_mp4, flv_files))

    print("\n✅ Conversion complete!")

if __name__ == "__main__":
    convert_all()

Streaming output truncated to the last 5000 lines.
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1066_TAI_FEA_XX.flv → outputMP4/1066_TAI_FEA_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1054_TIE_HAP_XX.flv → outputMP4/1054_TIE_HAP_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1076_TSI_SAD_XX.flv → outputMP4/1076_TSI_SAD_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1032_TSI_HAP_XX.flv → outputMP4/1032_TSI_HAP_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1046_IWL_SAD_XX.flv → outputMP4/1046_IWL_SAD_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1045_IOM_NEU_XX.flv → outputMP4/1045_IOM_NEU_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1001_TAI_NEU_XX.flv →

In [3]:
import os
import shutil
from tqdm import tqdm  # optional, for a progress bar

# Folder where all .flv files currently are
source_dir = "/content/outputMP4"     # change if needed
output_dir = "CREMA-D-Sorted1" # will be created automatically

# Mapping short emotion codes to full names
emotion_map = {
    "ANG": "Anger",
    "DIS": "Disgust",
    "FEA": "Fear",
    "HAP": "Happy",
    "NEU": "Neutral",
    "SAD": "Sad"
}

# Create target folders
for emotion_name in emotion_map.values():
    os.makedirs(os.path.join(output_dir, emotion_name), exist_ok=True)

# Loop over all videos
for filename in tqdm(os.listdir(source_dir)):
    if filename.endswith(".mp4"):
        parts = filename.replace(".mp4", "").split("_")
        if len(parts) == 4:
            emotion_code = parts[2]
            emotion_full = emotion_map.get(emotion_code)
            if emotion_full:
                src = os.path.join(source_dir, filename)
                dst = os.path.join(output_dir, emotion_full, filename)
                shutil.copy2(src, dst)  # or use shutil.move() if you want to move
        else:
            print(f"⚠️ Skipping malformed filename: {filename}")

print("✅ All videos rearranged by emotion!")


100%|██████████| 7442/7442 [00:01<00:00, 5741.73it/s]

✅ All videos rearranged by emotion!


In [4]:
import os
import cv2
from tqdm import tqdm

# Input: sorted emotion folders with videos
videos_root = "/content/CREMA-D-Sorted1"
# Output: matching folder structure for frames
frames_root = "/content/Df_Frames"
os.makedirs(frames_root, exist_ok=True)

for emotion in os.listdir(videos_root):
    emotion_dir = os.path.join(videos_root, emotion)
    if not os.path.isdir(emotion_dir):
        continue

    print(f"\n🎞️ Extracting frames for emotion: {emotion}")
    os.makedirs(os.path.join(frames_root, emotion), exist_ok=True)

    # Process each video file
    for video_file in tqdm(os.listdir(emotion_dir)):
        if not video_file.endswith((".mp4", ".flv")):
            continue

        video_path = os.path.join(emotion_dir, video_file)
        base_name = os.path.splitext(video_file)[0]
        out_dir = os.path.join(frames_root, emotion, base_name)
        os.makedirs(out_dir, exist_ok=True)

        vidcap = cv2.VideoCapture(video_path)
        if not vidcap.isOpened():
            print(f"⚠️ Could not open {video_path}")
            continue

        success, frame = vidcap.read()
        frame_idx = 0
        while success:
            frame_name = f"frame_{frame_idx:04d}.jpg"
            cv2.imwrite(os.path.join(out_dir, frame_name), frame)
            frame_idx += 1
            success, frame = vidcap.read()

        vidcap.release()
        print(f"✅ {video_file}: {frame_idx} frames extracted → {out_dir}")

print("\n✅ All videos processed and frames saved by emotion!")



🎞️ Extracting frames for emotion: Happy


  0%|          | 3/1271 [00:00<02:18,  9.14it/s]

✅ 1078_TAI_HAP_XX.mp4: 75 frames extracted → /content/Df_Frames/Happy/1078_TAI_HAP_XX
✅ 1046_TSI_HAP_XX.mp4: 49 frames extracted → /content/Df_Frames/Happy/1046_TSI_HAP_XX
✅ 1084_IEO_HAP_MD.mp4: 71 frames extracted → /content/Df_Frames/Happy/1084_IEO_HAP_MD


  0%|          | 6/1271 [00:00<02:04, 10.14it/s]

✅ 1034_MTI_HAP_XX.mp4: 91 frames extracted → /content/Df_Frames/Happy/1034_MTI_HAP_XX
✅ 1084_TIE_HAP_XX.mp4: 79 frames extracted → /content/Df_Frames/Happy/1084_TIE_HAP_XX
✅ 1016_MTI_HAP_XX.mp4: 77 frames extracted → /content/Df_Frames/Happy/1016_MTI_HAP_XX


...

✅ 1087_DFA_NEU_XX.mp4: 73 frames extracted → /content/Df_Frames/Neutral/1087_DFA_NEU_XX
✅ 1079_DFA_NEU_XX.mp4: 65 frames extracted → /content/Df_Frames/Neutral/1079_DFA_NEU_XX
✅ 1071_TAI_NEU_XX.mp4: 73 frames extracted → /content/Df_Frames/Neutral/1071_TAI_NEU_XX


100%|█████████▉| 1084/1087 [01:35<00:00, 12.54it/s]

✅ 1030_DFA_NEU_XX.mp4: 73 frames extracted → /content/Df_Frames/Neutral/1030_DFA_NEU_XX
✅ 1020_IWW_NEU_XX.mp4: 61 frames extracted → /content/Df_Frames/Neutral/1020_IWW_NEU_XX
✅ 1084_IEO_NEU_XX.mp4: 53 frames extracted → /content/Df_Frames/Neutral/1084_IEO_NEU_XX


100%|██████████| 1087/1087 [01:35<00:00, 11.35it/s]

✅ 1047_MTI_NEU_XX.mp4: 81 frames extracted → /content/Df_Frames/Neutral/1047_MTI_NEU_XX
✅ 1088_IEO_NEU_XX.mp4: 66 frames extracted → /content/Df_Frames/Neutral/1088_IEO_NEU_XX
✅ 1055_ITS_NEU_XX.mp4: 81 frames extracted → /content/Df_Frames/Neutral/1055_ITS_NEU_XX

✅ All videos processed and frames saved by emotion!


In [5]:
import os
import shutil
from tqdm import tqdm

# Root folder with current structure
frames_root = "/content/Df_Frames"  # contains subfolders per emotion, each with subfolders per video

# Loop over each emotion folder
for emotion in os.listdir(frames_root):
    emotion_dir = os.path.join(frames_root, emotion)
    if not os.path.isdir(emotion_dir):
        continue

    print(f"\n🎞️ Merging frames for emotion: {emotion}")
    count = 0

    # Loop through subfolders (each video)
    for video_folder in tqdm(os.listdir(emotion_dir)):
        video_path = os.path.join(emotion_dir, video_folder)
        if not os.path.isdir(video_path):
            continue

        # Move each frame into the parent emotion folder
        for frame_file in os.listdir(video_path):
            if not frame_file.lower().endswith((".jpg", ".png")):
                continue

            src = os.path.join(video_path, frame_file)
            # Rename frame to include video folder name (so no duplicates)
            new_name = f"{video_folder}_{frame_file}"
            dst = os.path.join(emotion_dir, new_name)

            shutil.move(src, dst)
            count += 1

        # Remove now-empty subfolder
        os.rmdir(video_path)

    print(f"✅ {emotion}: {count} frames merged into one folder")

print("\n✅ All emotion folders flattened successfully!")



🎞️ Merging frames for emotion: Happy


100%|██████████| 1271/1271 [00:02<00:00, 585.86it/s]


✅ Happy: 90665 frames merged into one folder

🎞️ Merging frames for emotion: Disgust


100%|██████████| 1271/1271 [00:02<00:00, 484.67it/s]


✅ Disgust: 109272 frames merged into one folder

🎞️ Merging frames for emotion: Fear


...

✅ Anger: 99983 frames merged into one folder

🎞️ Merging frames for emotion: Sad


100%|██████████| 1271/1271 [00:02<00:00, 509.42it/s]


✅ Sad: 100530 frames merged into one folder

🎞️ Merging frames for emotion: Neutral


100%|██████████| 1087/1087 [00:01<00:00, 556.61it/s]

✅ Neutral: 80309 frames merged into one folder

✅ All emotion folders flattened successfully!


In [6]:
import os
from PIL import Image
from collections import Counter

# Path to your merged dataset folder (update as needed)
frames_root = "/content/Df_Frames"

sizes = Counter()

# Loop through emotion folders
for emotion in os.listdir(frames_root):
    emotion_dir = os.path.join(frames_root, emotion)
    if not os.path.isdir(emotion_dir):
        continue

    # Loop through all image files
    for file in os.listdir(emotion_dir):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(emotion_dir, file)
            try:
                with Image.open(path) as img:
                    sizes[img.size] += 1  # img.size returns (width, height)
            except Exception as e:
                print(f"Error reading {path}: {e}")

# Display unique sizes and counts
print("\nUnique image sizes found:")
for size, count in sizes.items():
    print(f"{size}: {count} images")

# Optional: check if all are consistent
if len(sizes) == 1:
    print("\n✅ All images have the same size!")
else:
    print("\n⚠️ Images have different sizes! You may need to resize them for training.")



Unique image sizes found:
(480, 360): 577982 images

✅ All images have the same size!


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Path to your existing dataset (each subfolder = emotion class)
frames_root = "/content/Df_Frames"

# Create an ImageDataGenerator with realistic emotion augmentations
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)

# Loop through each emotion folder
for emotion in os.listdir(frames_root):
    emotion_dir = os.path.join(frames_root, emotion)
    if not os.path.isdir(emotion_dir):
        continue

    # Set up a generator that reads from this folder
    gen = datagen.flow_from_directory(
        frames_root,
        classes=[emotion],             # only this class
        class_mode=None,
        target_size=(224, 224),
        batch_size=1,
        save_to_dir=emotion_dir,       # same folder — will merge automatically
        save_prefix='aug',
        save_format='jpg'
    )

    # Generate N augmented images for this class
    N = 10000  # adjust as you need per class
    for i in range(N):
        next(gen)

    print(f"✅ Generated {N} new images for class: {emotion}")


Found 91165 images belonging to 1 classes.
✅ Generated 10000 new images for class: Happy
Found 109772 images belonging to 1 classes.
✅ Generated 10000 new images for class: Disgust
Found 97723 images belonging to 1 classes.
✅ Generated 10000 new images for class: Fear
Found 100483 images belonging to 1 classes.
✅ Generated 10000 new images for class: Anger
Found 101030 images belonging to 1 classes.
✅ Generated 10000 new images for class: Sad
Found 80809 images belonging to 1 classes.
✅ Generated 10000 new images for class: Neutral


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

# =========================================
# CONFIGURATION
# =========================================
frames_root = "/content/Df_Frames"     # Path to your dataset (each emotion in subfolder)
input_shape = (360, 480, 3)            # Height, Width, Channels
latent_dim = 256
batch_size = 256
epochs = 30
save_dir = "/content/autoencoder_emotion"
os.makedirs(save_dir, exist_ok=True)

# =========================================
# DATA GENERATOR (with denoising)
# =========================================
base_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.1
)

train_gen = base_datagen.flow_from_directory(
    frames_root,
    target_size=input_shape[:2],  # (360, 480)
    batch_size=batch_size,
    class_mode=None,
    subset='training',
    shuffle=True
)

val_gen = base_datagen.flow_from_directory(
    frames_root,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode=None,
    subset='validation'
)

# Add Gaussian noise dynamically
def noisy_gen(generator, noise_factor=0.08):
    for imgs in generator:
        noise = np.random.normal(0, noise_factor, imgs.shape)
        noisy = np.clip(imgs + noise, 0., 1.)
        yield noisy, imgs

train_noisy = noisy_gen(train_gen)
val_noisy = noisy_gen(val_gen)

steps_train = int(np.ceil(train_gen.samples / batch_size))
steps_val = int(np.ceil(val_gen.samples / batch_size))

# =========================================
# AUTOENCODER MODEL
# =========================================
def build_emotion_autoencoder(input_shape=(360,480,3), latent_dim=256):
    inp = layers.Input(shape=input_shape)

    # ===== Encoder =====
    x = layers.Conv2D(32, 3, strides=2, padding='same')(inp)  # 180x240
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(64, 3, strides=2, padding='same')(x)    # 90x120
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Residual Block
    res = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    res = layers.Conv2D(64, 3, padding='same')(res)
    x = layers.Add()([x, res])
    x = layers.ReLU()(x)

    x = layers.Conv2D(128, 3, strides=2, padding='same')(x)   # 45x60
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(256, 3, strides=2, padding='same')(x)   # 23x30 approx
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Flatten to latent vector
    x = layers.GlobalAveragePooling2D()(x)
    latent = layers.Dense(latent_dim, name="latent_vector")(x)

    # ===== Decoder =====
    x = layers.Dense(23 * 30 * 256)(latent)
    x = layers.Reshape((23, 30, 256))(x)

    x = layers.Conv2DTranspose(128, 3, strides=2, padding='same', activation='relu')(x)  # 46x60
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same', activation='relu')(x)   # 92x120
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(32, 3, strides=2, padding='same', activation='relu')(x)   # 184x240
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(16, 3, strides=2, padding='same', activation='relu')(x)   # 368x480
    x = layers.BatchNormalization()(x)

    # Crop slightly to match 360 height exactly
    x = layers.Cropping2D(((4, 4), (0, 0)))(x)  # from 368x480 → 360x480

    out = layers.Conv2DTranspose(3, 3, activation='sigmoid', padding='same')(x)

    autoencoder = Model(inp, out, name="emotion_autoencoder")
    encoder = Model(inp, latent, name="encoder_for_LSTM")

    return autoencoder, encoder

autoencoder, encoder = build_emotion_autoencoder(input_shape, latent_dim)
autoencoder.summary()

# =========================================
# COMPILE & TRAIN
# =========================================
autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='mse',
    metrics=['mae']
)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(save_dir, "best_autoencoder.h5"),
        monitor="val_loss",
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=3, verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True, verbose=1
    )
]

history = autoencoder.fit(
    train_noisy,
    validation_data=val_noisy,
    steps_per_epoch=steps_train,
    validation_steps=steps_val,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

# =========================================
# SAVE MODELS
# =========================================
autoencoder.save(os.path.join(save_dir, "final_autoencoder.h5"))
encoder.save(os.path.join(save_dir, "final_encoder.h5"))

print("✅ Training complete — Models saved in:", save_dir)


Found 576886 images belonging to 6 classes.
Found 64096 images belonging to 6 classes.


Model: "emotion_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 360, 480,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 180, 240,  │        896 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 180, 240,  │        128 │ conv2d_18[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_15 (ReLU)     │ (None, 180, 240,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 90, 120,   │     18,496 │ re_lu_15[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 90, 120,   │        256 │ conv2d_19[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_16 (ReLU)     │ (None, 90, 120,   │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 90, 120,   │     36,928 │ re_lu_16[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 90, 120,   │     36,928 │ conv2d_20[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 90, 120,   │          0 │ re_lu_16[0][0],   │
│                     │ 64)               │            │ conv2d_21[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_17 (ReLU)     │ (None, 90, 120,   │          0 │ add_3[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 45, 60,    │     73,856 │ re_lu_17[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 45, 60,    │        512 │ conv2d_22[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_18 (ReLU)     │ (None, 45, 60,    │          0 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 23, 30,    │    295,168 │ re_lu_18[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 23, 30,    │      1,024 │ conv2d_23[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_19 (ReLU)     │ (None, 23, 30,    │          0 │ batch_normalizat

 Total params: 46,319,779 (176.70 MB)

 Trainable params: 46,318,339 (176.69 MB)

 Non-trainable params: 1,440 (5.62 KB)

Epoch 1/30
